### Importing Required Libraries
- pandas for data manipulation
- numpy for numerical operations
- scikit-learn for machine learning
- XGBoost for gradient boosting


In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

c:\Users\Tanay\miniconda3\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading Training Dataset
Loading the training data for model development


In [3]:
train_df = pd.read_csv("assets/final_preprocessed_train.csv")
train_df

,Unnamed: 0,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,SaleType,SaleCondition,SalePrice,AgeWhenSold,RenovatedAgeSold
0,0,60,3,65.0,-0.207071,1,0,3,3,0,...,0,0,0,-0.087658,2,8,4,208500,5,5
1,1,20,3,80.0,-0.091855,1,0,3,3,0,...,0,0,0,-0.087658,5,8,4,181500,31,31
2,2,60,3,68.0,0.073455,1,0,0,3,0,...,0,0,0,-0.087658,9,8,4,223500,7,6
3,3,70,3,60.0,-0.096864,1,0,0,3,0,...,0,0,0,-0.087658,2,8,0,140000,91,36
4,4,60,3,84.0,0.375020,1,0,0,3,0,...,0,0,0,-0.087658,12,8,4,250000,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,1455,60,3,62.0,-0.260471,1,0,3,3,0,...,0,0,0,-0.087658,8,8,4,175000,8,7
1091,1456,20,3,85.0,0.266316,1,0,3,3,0,...,0,3,0,-0.087658,2,8,4,210000,32,22
1092,1457,70,3,66.0,-0.147760,1,0,3,3,0,...,0,1,3,4.951415,5,8,4,266500,69,4
1093,1458,20,3,68.0,-0.080133,1,0,3,3,0,...,0,0,0,-0.087658,4,8,4,142125,60,14


### Neural Network Model
Training a neural network model


In [4]:
train_df.drop(columns=["Unnamed: 0"], inplace=True)
y = train_df["SalePrice"]
X = train_df.drop(columns=["SalePrice"])

### Model Training
Training the machine learning model


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Training
Training the machine learning model


In [6]:
y_train

6       307000
575     127500
821     146000
1063    122500
905     117000
         ...  
330     119000
466     359100
121     215000
1044    246578
860     149000
Name: SalePrice, Length: 876, dtype: int64

## Check for GPU


### Importing Required Libraries
- pandas for data manipulation
- numpy for numerical operations
- scikit-learn for machine learning


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Normalize input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

# Convert Pandas DataFrames to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Define the improved regression model
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.hidden1 = nn.Linear(input_size, 20)  # Increased neurons
        self.hidden2 = nn.Linear(20, 10)  # Additional hidden layer
        self.output = nn.Linear(10, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

# Hyperparameters (updated)
l1_lambda = 0.001
l2_lambda = 0.001
learning_rate = 0.001
epochs = 500  # More training

# Define model and move it to GPU
input_size = X_train_torch.shape[1]
model = RegressionNN(input_size).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with GPU
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train_torch)
    
    # Compute base MSE loss
    mse_loss = criterion(y_pred, y_train_torch)
    rmse_loss = torch.sqrt(mse_loss)

    # Compute L1 and L2 loss
    l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
    l2_loss = sum(torch.sum(param ** 2) for param in model.parameters())

    # Total loss with regularization
    # total_loss = mse_loss + l1_lambda * l1_loss + l2_lambda * l2_loss

    # Backpropagation
    rmse_loss.backward()
    optimizer.step()

    if epoch % 50 == 0:  # Less frequent logging
        print(f"Epoch {epoch}, RMSE: {rmse_loss.item()}")

# Evaluate model on test data
with torch.no_grad():
    y_test_pred = model(X_test_torch)
    test_mse_loss = criterion(y_test_pred, y_test_torch)
    test_rmse_loss = torch.sqrt(test_mse_loss)
    print(f"Test RMSE: {test_rmse_loss.item()}")


Using device: cuda
Epoch 0, RMSE: 203734.984375
Epoch 50, RMSE: 203733.828125
Epoch 100, RMSE: 203727.71875
Epoch 150, RMSE: 203707.875
Epoch 200, RMSE: 203664.25
Epoch 250, RMSE: 203587.484375
Epoch 300, RMSE: 203468.6875
Epoch 350, RMSE: 203299.46875
Epoch 400, RMSE: 203071.890625
Epoch 450, RMSE: 202778.46875
Test RMSE: 207442.875


### Importing Required Libraries
- numpy for numerical operations
- scikit-learn for machine learning


## Normalize input data


### Neural Network Model
Training a neural network model


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Normalize input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

# Convert Pandas DataFrames to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Define the regression model
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.hidden1 = nn.Linear(input_size, 20)  
        self.hidden2 = nn.Linear(20, 10)  
        self.output = nn.Linear(10, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

# Define model and move it to GPU
input_size = X_train_torch.shape[1]
model = RegressionNN(input_size).to(device)

# Define loss function
criterion = nn.MSELoss()

# Use L-BFGS optimizer
optimizer = optim.LBFGS(model.parameters(), lr=0.1, max_iter=20)

# Training loop with L-BFGS
epochs = 250
for epoch in range(epochs):
    def closure():
        optimizer.zero_grad()  # Clear previous gradients
        y_pred = model(X_train_torch)
        
        # Compute MSE loss
        mse_loss = criterion(y_pred, y_train_torch)
        rmse_loss = torch.sqrt(mse_loss)
        
        # Compute L1 and L2 loss
        l1_lambda = 0.001
        l2_lambda = 0.001
        l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
        l2_loss = sum(torch.sum(param ** 2) for param in model.parameters())
        
        # Total loss with regularization
        # total_loss = mse_loss + l1_lambda * l1_loss + l2_lambda * l2_loss
        
        rmse_loss.backward()  # Compute gradients
        return rmse_loss

    optimizer.step(closure)  # Perform L-BFGS step

    if epoch % 50 == 0:
        with torch.no_grad():
            y_pred_train = model(X_train_torch)
            train_mse_loss = criterion(y_pred_train, y_train_torch)
            train_rmse_loss = torch.sqrt(train_mse_loss)
            print(f"Epoch {epoch}, Train RMSE: {train_rmse_loss.item()}")

# Evaluate model on test data
with torch.no_grad():
    y_test_pred = model(X_test_torch)
    test_mse_loss = criterion(y_test_pred, y_test_torch)
    test_rmse_loss = torch.sqrt(test_mse_loss)
    print(f"Test RMSE: {test_rmse_loss.item()}")


Using device: cuda
Epoch 0, Train RMSE: 203734.421875
Epoch 50, Train RMSE: 928.4501342773438
Epoch 100, Train RMSE: 1.6931579113006592


KeyboardInterrupt: 

## Using L1 norm


### Neural Network Model
Training a neural network model


In [12]:
# Using L1 norm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Normalize input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

# Convert Pandas DataFrames to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Define the regression model
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.hidden1 = nn.Linear(input_size, 20)  
        self.hidden2 = nn.Linear(20, 10)  
        self.output = nn.Linear(10, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

# Define model and move it to GPU
input_size = X_train_torch.shape[1]
model = RegressionNN(input_size).to(device)

# Define loss function
criterion = nn.MSELoss()

# Use L-BFGS optimizer
optimizer = optim.LBFGS(model.parameters(), lr=0.1, max_iter=20)

# Training loop with L-BFGS
epochs = 100
for epoch in range(epochs):
    def closure():
        optimizer.zero_grad()  # Clear previous gradients
        y_pred = model(X_train_torch)
        
        # Compute MSE loss
        mse_loss = criterion(y_pred, y_train_torch)
        rmse_loss = torch.sqrt(mse_loss)
        
        # Compute L1 and L2 loss
        l1_lambda = 0.001
        l2_lambda = 0.001
        l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
        l2_loss = sum(torch.sum(param ** 2) for param in model.parameters())
        
        # Total loss with regularization
        total_loss = mse_loss + l1_lambda * l1_loss
        
        total_loss.backward()  # Compute gradients
        return total_loss

    optimizer.step(closure)  # Perform L-BFGS step

    if epoch % 50 == 0:
        with torch.no_grad():
            y_pred_train = model(X_train_torch)
            train_mse_loss = criterion(y_pred_train, y_train_torch)
            train_rmse_loss = torch.sqrt(train_mse_loss)
            print(f"Epoch {epoch}, Train RMSE: {train_rmse_loss.item()}")

# Evaluate model on test data
with torch.no_grad():
    y_test_pred = model(X_test_torch)
    test_mse_loss = criterion(y_test_pred, y_test_torch)
    test_rmse_loss = torch.sqrt(test_mse_loss)
    print(f"Test RMSE: {test_rmse_loss.item()}")


Using device: cuda
Epoch 0, Train RMSE: 203734.8125
Epoch 50, Train RMSE: 2279.405029296875
Test RMSE: 69300.046875


## Using L2 norm


### Neural Network Model
Training a neural network model


In [47]:
# Using L2 norm


# Normalize input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

# Convert Pandas DataFrames to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Define the regression model
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.hidden1 = nn.Linear(input_size, 20)  
        self.hidden2 = nn.Linear(20, 10)  
        self.output = nn.Linear(10, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

# Define model and move it to GPU
input_size = X_train_torch.shape[1]
model = RegressionNN(input_size).to(device)

# Define loss function
criterion = nn.MSELoss()

# Use L-BFGS optimizer
optimizer = optim.LBFGS(model.parameters(), lr=0.1, max_iter=20)

# Training loop with L-BFGS
epochs = 250
for epoch in range(epochs):
    def closure():
        optimizer.zero_grad()  # Clear previous gradients
        y_pred = model(X_train_torch)
        
        # Compute MSE loss
        mse_loss = criterion(y_pred, y_train_torch)
        rmse_loss = torch.sqrt(mse_loss)
        
        # Compute L1 and L2 loss
        l1_lambda = 0.001
        l2_lambda = 0.001
        l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
        l2_loss = sum(torch.sum(param ** 2) for param in model.parameters())
        
        # Total loss with regularization
        total_loss = mse_loss + l2_lambda * l2_loss
        
        total_loss.backward()  # Compute gradients
        return total_loss

    optimizer.step(closure)  # Perform L-BFGS step

    if epoch % 50 == 0:
        with torch.no_grad():
            y_pred_train = model(X_train_torch)
            train_mse_loss = criterion(y_pred_train, y_train_torch)
            train_rmse_loss = torch.sqrt(train_mse_loss)
            print(f"Epoch {epoch}, Train RMSE: {train_rmse_loss.item()}")

# Evaluate model on test data
with torch.no_grad():
    y_test_pred = model(X_test_torch)
    test_mse_loss = criterion(y_test_pred, y_test_torch)
    test_rmse_loss = torch.sqrt(test_mse_loss)
    print(f"Test RMSE: {test_rmse_loss.item()}")


Epoch 0, Train RMSE: 255189008384.0
Epoch 50, Train RMSE: 92527.921875
Epoch 100, Train RMSE: 87112.78125
Epoch 150, Train RMSE: 86264.5546875
Epoch 200, Train RMSE: 84034.2734375
Test RMSE: 20970590.0
